In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# pass rescale to normalize data 
train_datagen = ImageDataGenerator(rescale=1./255)

# always point to main directory and not subdirectories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),    # images all need to be the same size to train
    batch_size=128,
    class_mode='binary'
)

In [ ]:
# pass rescale to normalize data 
test_datagen = ImageDataGenerator(rescale=1./255)

# always point to main directory and not subdirectories
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(300, 300),    # images all need to be the same size to train
    batch_size=128,
    class_mode='binary'
)

In [ ]:
model = tf.keras.mode.Sequential([
    # convolutions and pooling
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])